In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("lrex").getOrCreate()

# Load training data
data = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/necmettinceylan@hotmail.com/Ecommerce_Customers.csv", 
                                     inferSchema=True, header=True)

In [0]:
  data.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [0]:
data.head(1)

Out[3]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [0]:
for item in data.head(1)[0]:
  print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
data.columns

Out[6]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
assembler = VectorAssembler( inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'], outputCol="features" )

output = assembler.transform(data)

In [0]:
output.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
final_data = output.select( "features", "Yearly Amount Spent" )
final_data.show()

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
[33.0009147556426...| 487.54750486747207|
[34.3055566297555...| 581.8523440352177|
[33.3306725236463...| 599.4060920457634|
[33.8710378793419...| 637.102447915074|
[32.0215955013870...| 521.5721747578274|
[32.7391429383803...| 549.9041461052942|
[33.9877728956856...| 570.2004089636196|
[31.9365486184489...| 427.1993848953282|
[33.9925727749537...| 492.6060127179966|
[33.8793608248049...| 522.3374046069357|
[29.5324289670579...| 408.6403510726275|
[33.1903340437226...| 573.4158673313865|
[32.3879758531538...| 470.4527333009554|
[30.7377203726281...| 461.7807421962299|
[32.1253868972878...| 457.84769594494855|
[32.3388993230671...| 407.70454754954415|
[32.1878120459321...| 452.3156754800354|
[32.6178560628234...| 605.061038804892|
+--------------------+-------------------+
only showing top 20 rows

In [0]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [0]:
test_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 148|
 mean| 504.5617727783442|
 stddev| 78.14542837690476|
 min| 304.1355915788555|
 max| 744.2218671047146|
+-------+-------------------+

In [0]:
lr = LinearRegression( labelCol="Yearly Amount Spent" )

In [0]:
lr_model = lr.fit(train_data)


In [0]:
test_results = lr_model.evaluate( test_data )
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
-11.792463081731853|
0.23520561874437362|
-3.9468665564829166|
 -6.58462179175973|
-22.199137067548122|
 19.486812204378623|
 3.018458472485406|
 -7.286558066987141|
 -18.21206937976558|
-14.705797087041674|
 16.578749393427017|
 6.746204748691923|
-2.9271081373403263|
 7.491344576481595|
-5.9171423662123175|
 11.865249957727485|
 5.345840268496772|
 8.222308656883001|
 4.970772037404231|
0.07429618111171976|
+-------------------+
only showing top 20 rows

In [0]:
test_results.rootMeanSquaredError

Out[17]: 9.874056747432167

In [0]:
test_results.r2

Out[18]: 0.9839258333771491

In [0]:
final_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [0]:
test_data.show()

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[30.3931845423455...| 319.9288698031936|
[30.5743636841713...| 442.06441375806565|
[30.8794843441274...| 490.2065999848547|
[31.0613251567161...| 487.5554580579016|
[31.1239743499119...| 486.9470538397658|
[31.3123495994443...| 463.5914180279406|
[31.4459724827577...| 484.87696493512857|
[31.4474464941278...| 418.602742095224|
[31.5702008293202...| 545.9454921414049|
[31.5741380228732...| 544.4092721605869|
[31.6098395733896...| 444.54554965110816|
[31.6548096756927...| 475.2634237275485|
[31.8186165667690...| 446.41867337013565|
[31.8209982016720...| 424.67528101321335|
[31.8745516945853...| 392.2852442462675|
[31.9262720263601...| 392.2049334443264|
[31.9480174211613...| 461.9208768928978|
[31.9549038566348...| 439.997879939927|
[31.9764800614612...| 330.5944460341002|
[32.0047530203648...| 463.74598112062944|
+--------------------+-------------------+
only showing top 20 rows